In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import class_weight # ★ 딥러NING에서 불균형 데이터 처리를 위해 추가

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# --- 0. 데이터 불러오기 ---
file_name = '../../DataPreprocessing/earthquake/3차 전처리 - 단층 이동 데이터/TEST_train_data_final_with_fault_counts_past_year_full_month.csv'
try:
    df = pd.read_csv(file_name)
except FileNotFoundError:
    print(f"오류: '{file_name}' 파일을 찾을 수 없습니다.")
    exit()

print(f"'{file_name}' 파일 (총 {len(df)}행)을 성공적으로 불러왔습니다.")

# --- 1. 전처리 ---
if 'is_ocean' in df.columns:
    df['is_ocean'] = df['is_ocean'].apply(lambda x: 1 if (x == True or str(x).lower() == 'true') else 0)
if 'is_steep_slope' in df.columns:
    df['is_steep_slope'] = df['is_steep_slope'].apply(lambda x: 1 if (x == True or str(x).lower() == 'true') else 0)

# --- 2. 특성(X)과 타겟(y) 정의 ---
feature_names = [
    'magnitude',
    'depth',
    'is_ocean',
    'is_steep_slope',
    'horizontal_count_1y_full',
    'vertical_count_1y_full'
]
df_model = df[feature_names + ['tsunami']].dropna()
X = df_model[feature_names]
y = df_model['tsunami']

print(f"\n총 {len(df_model)}개의 데이터로 모델링을 시작합니다.")

# --- 3. 훈련 / 테스트 데이터 분리 ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"훈련 데이터: {len(X_train)}개 / 테스트 데이터: {len(X_test)}개")

# --- 4. 데이터 스케일링 ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ★ (추가) 불균형 데이터 가중치 계산 (로지스틱의 class_weight='balanced'와 동일한 역할)
# 0(쓰나미X)이 1(쓰나미O)보다 많으므로, 1에 더 높은 가중치를 줌
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {0: weights[0], 1: weights[1]}
print(f"클래스 가중치 (0: No, 1: Yes): {class_weights_dict}")


# --- 5. ★★★ 딥러닝 모델 구축 (Keras) ★★★ ---
print("\n--- 1. 딥러닝 모델 구축 중... ---")

model = Sequential()
# 입력층 (6개 특성) -> 16개 노드로
model.add(Dense(16, activation='relu', input_shape=(X_train_scaled.shape[1],)))
# 과적합 방지를 위한 Dropout (노드 50% 비활성화)
model.add(Dropout(0.5))
# 은닉층 (16개 -> 8개 노드로)
model.add(Dense(8, activation='relu'))
# 출력층 (1개 노드, 0~1 사이 확률 출력)
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일 (학습 방식 설정)
model.compile(
    optimizer='adam',                # 옵티마이저
    loss='binary_crossentropy',      # 0/1 분류 손실 함수
    metrics=['accuracy']             # 평가 지표
)
model.summary() # 모델 구조 출력

# --- 6. ★★★ 딥러닝 모델 학습 ★★★ ---
print("\n--- 2. 딥러닝 모델 학습 중... ---")
# 50번 반복 학습
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=50,                       # 50번 반복 학습
    batch_size=32,                   # 32개씩 묶어서 학습
    validation_data=(X_test_scaled, y_test), # 매번 테스트셋으로 검증
    class_weight=class_weights_dict, # 불균형 가중치 적용
    verbose=0                        # 학습 과정 상세 출력 (1로 바꾸면 다 보임)
)
print("학습 완료.")

# --- 7. ★★★ 딥러닝 모델 평가 ★★★ ---
print("\n--- 3. 딥러닝 모델 평가 (테스트 데이터) ---")
# 테스트 데이터로 최종 성능 평가
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"모델 정확도(Accuracy): {accuracy * 100:.2f}%")

# 예측 (0.5 이상이면 1, 아니면 0)
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)

# 분류 리포트 (로지스틱 회귀와 동일)
print("\n[분류 리포트 (Classification Report)]")
print(classification_report(y_test, y_pred, target_names=['No Tsunami (0)', 'Tsunami (1)']))

# 혼동 행렬 (로지스틱 회귀와 동일)
print("\n[혼동 행렬 (Confusion Matrix)]")
cm = confusion_matrix(y_test, y_pred)
print(cm)

'../../DataPreprocessing/earthquake/3차 전처리 - 단층 이동 데이터/TEST_train_data_final_with_fault_counts_past_year_full_month.csv' 파일 (총 782행)을 성공적으로 불러왔습니다.

총 782개의 데이터로 모델링을 시작합니다.
훈련 데이터: 625개 / 테스트 데이터: 157개
클래스 가중치 (0: No, 1: Yes): {0: np.float64(0.8180628272251309), 1: np.float64(1.286008230452675)}

--- 1. 딥러닝 모델 구축 중... ---


C:\Python1\Python313\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257 (1.00 KB)

 Trainable params: 257 (1.00 KB)

 Non-trainable params: 0 (0.00 B)


--- 2. 딥러닝 모델 학습 중... ---
학습 완료.

--- 3. 딥러닝 모델 평가 (테스트 데이터) ---
모델 정확도(Accuracy): 59.87%
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 

[분류 리포트 (Classification Report)]
                precision    recall  f1-score   support

No Tsunami (0)       0.75      0.52      0.61        96
   Tsunami (1)       0.49      0.72      0.58        61

      accuracy                           0.60       157
     macro avg       0.62      0.62      0.60       157
  weighted avg       0.65      0.60      0.60       157


[혼동 행렬 (Confusion Matrix)]
[[50 46]
 [17 44]]
